In [ ]:
import os
import glob
import obspy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.DataFrame()
df['channels'] = ['VDT', 'VPB', 'GLA', 'GLO', 'GEL', 'VM1', 'VM2', 'VM3', 'VM4', 'VM5', 'VM6', \
                    'VEC', 'VEI', 'VCO', 'GAN', 'LCE', 'GPL', 'LCQ', 'GST', 'GNS']
df['labels'] = ['temperature\n(C)', 'buffer\n%', 'longitude', 'latitude', 'elevation\n(m)', \
                'A-Z', 'A-N', 'A-E', 'B-Z', 'B-N', 'B-E', 'current\n(A)', 'input\nvoltage\n(V)', 'voltage\n(V)', \
                'antenna\nstatus', 'clock\nerror\n(s)', 'locking', 'clock\nquality\n%', 'status', 'num\nsatellites'    ]
df['multipliers'] = [1e-3, 0.1, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-6, 1e-3, 1e-3, 1e-3, 1, 1e-3, 1, 1, 1, 1 ]
#display(df)

def get_averages(st, df):
    avdict = {}
    avdict['date'] = pd.to_datetime(f'{st[0].stats.starttime}')
    for tr in st:
        row = df[df['channels']==tr.stats.channel].iloc[0]
        chomplabel = row['labels'].replace('\n', '')
        avdict[chomplabel] = np.mean(tr.data * row['multipliers'])
    #display(avdict)
    return avdict

def plot_soh(st, df):

    main_channels = ['VDT', 'VEC', 'VEI', 'VCO', 'LCE', 'LCQ', 'GNS']

    fh, axh = plt.subplots(len(main_channels))
    count = 0
    for channel in df['channels']:

        if not channel in main_channels:
            continue

        tr = st.select(channel=channel)[0]
        row = df[df['channels']==tr.stats.channel].iloc[0]
        chomplabel = row['labels'].replace('\n', '')        
        axh[count].plot(tr.times()/3600, tr.data * row['multipliers'])
        axh[count].set_ylabel(row['labels'])
        axh[count].grid()
        axh[count].set_xlim(0,24)
        if count == len(main_channels)-1:
            axh[count].set_xlabel('Hours')
        
        count += 1

    fh.suptitle(f'SOH data on {tr.stats.starttime.strftime("%Y-%m-%d")}') 


def plot_soh_year(station, year, station_year_soh_df, df):

    for i, main_channels in enumerate([['VDT', 'VEC', 'VEI', 'VCO', 'LCE', 'LCQ', 'GNS'], ['VPB', 'GLA', 'GLO', 'GEL'], ['VM1', 'VM2', 'VM3', 'VM4', 'VM5', 'VM6'], \
                    ['GAN', 'GPL', 'GST']]):
        main_labels=[]
        for channel in main_channels:
            row = df[df['channels']==channel].iloc[0]
            main_labels.append(row['labels'])

        fh, axh = plt.subplots(len(main_channels))
        for count, col in enumerate(main_labels):
            chomplabel = col.replace('\n', '')        
            station_year_soh_df.plot(ax=axh[count], x='date', y=chomplabel, legend=False)
            axh[count].set_ylabel(col)
            if count < len(main_channels)-1:
                axh[count].set_xlabel('')
                axh[count].set_xticklabels([])

        fh.suptitle(f"SOH data plot {i} for {station} for {year}") 
        plt.show()

sdsdir = '/data/SDS'
year='2022'
network = 'FL'
#dict_of_df = dict()
for stationdir in sorted(glob.glob(os.path.join(sdsdir, year, network, '*'))):
    print(stationdir)
    station = os.path.basename(stationdir)
    filedir = os.path.join(stationdir, 'SOH.S')
    if os.path.isdir(filedir):
        lod = []
        for juldayint in range(367):
            julday = f'{juldayint:03d}'

            sdsfile = os.path.join(filedir, f'{network}.{station}.D0.SOH.S.{year}.{julday}.miniseed')
            if not os.path.isfile(sdsfile): 
                sdsfile = os.path.join(filedir, f'{network}.{station}.D0.SOH.S.{year}.{julday}')
            if os.path.isfile(sdsfile):
                #print(sdsfile) 
                st = obspy.read(sdsfile, format='MSEED')
                #st.plot(equal_scale=False);
                #plot_soh(st, df)
                avdict = get_averages(st, df)
                lod.append(avdict)
        station_year_soh_df = pd.DataFrame(lod)
        csvfile = f'soh_{network}_{year}_{station}.csv'
        station_year_soh_df.round(5).to_csv(csvfile)

        plot_soh_year(station, year, station_year_soh_df, df)
